In [21]:
# import requests
# from bs4 import BeautifulSoup

# headers = {
#              'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.124 Mobile Safari/537.36 Edg/102.0.1245.44'
# }
# LoginData = {
    
    
#     'ctl00_MainContent_ToolkitScriptManager1_HiddenField': ";;AjaxControlToolkit, Version=4.1.51116.0, Culture=neutral, PublicKeyToken=28f01b0e84b6d53e:en-US:fd384f95-1b49-47cf-9b47-2fa2a921a36a:475a4ef5:addc6819:5546a2b:d2e10b12:effe2a26:37e2e5c9:5a682656:c7029a2:e9e598a9",
#     'ctl00$MainContent$Ddl_Rpt_type': 'Retail',
#     'ctl00$MainContent$ddl_Language' :'English',
#     'ctl00$MainContent$Rbl_Rpt_type': 'Price report',
#     'ctl00$MainContent$Ddl_Rpt_Option0': 'Daily Prices',
#     'ctl00$MainContent$Txt_FrmDate': '25/07/2022',
#     'ctl00$MainContent$btn_getdata1': 'Get Data',
# }
# with requests.Session() as s:
#     url = "https://fcainfoweb.nic.in/reports/report_menu_web.aspx"
#     url1="https://fcainfoweb.nic.in/reports/Report_daily1_Web_New.aspx"
#     r = s.get(url, headers=headers)
#     soup = BeautifulSoup(r.content, 'html.parser')
#     #print(soup)
#     LoginData['__VIEWSTATE'] = soup.find('input', attrs={'id': '__VIEWSTATE'})['value']
#     LoginData['__VIEWSTATEGENERATOR'] = soup.find('input', attrs={'id': '__VIEWSTATEGENERATOR'})['value']
#     LoginData['__EVENTVALIDATION'] = soup.find('input', attrs={'id': '__EVENTVALIDATION'})['value']
    
#     r1 = s.post(url, data=LoginData, headers=headers, allow_redirects=True)
#     r2=  s.get(url1,data=LoginData, headers=headers)
#     #soup1 = BeautifulSoup(r2, 'html.parser')
    


In [22]:
import numpy as np
import requests
from   bs4 import BeautifulSoup
import pandas as pd

In [36]:
url = 'https://fcainfoweb.nic.in/reports/report_menu_web.aspx'

headers = { 'User-Agent': 'Mozilla/5.0' }

s = requests.Session()
s.headers.update(headers)

data = {
    '__EVENTTARGET': 'ctl00$MainContent$Rbl_Rpt_type$0',
    '__VIEWSTATEENCRYPTED': '',
    'ctl00$MainContent$Ddl_Rpt_type': 'Retail',
    'ctl00$MainContent$ddl_Language': 'English',
    'ctl00$MainContent$Rbl_Rpt_type': 'Price report',
}

In [37]:
def update_data():
    soup = BeautifulSoup(r.content, 'html.parser')
    data['__VIEWSTATE']          = soup.find('input', attrs={'id': '__VIEWSTATE'})['value']
    data['__VIEWSTATEGENERATOR'] = soup.find('input', attrs={'id': '__VIEWSTATEGENERATOR'})['value']
    data['__EVENTVALIDATION']    = soup.find('input', attrs={'id': '__EVENTVALIDATION'})['value']
    
    

In [38]:
def get_report_html(date):
    r = s.get(url)
    update_data()

    """Select Price Report"""
    r = s.post(url, data=data)
    update_data()

    """Select Daily Prices"""
    data['__EVENTTARGET'] = 'ctl00$MainContent$Ddl_Rpt_Option0'
    data['ctl00$MainContent$Ddl_Rpt_Option0'] = 'Daily Prices'

    r = s.post(url, data=data)
    update_data()

    data['__EVENTTARGET'] = ''
    data['ctl00$MainContent$Txt_FrmDate']  = date
    data['ctl00$MainContent$btn_getdata1'] =  'Get Data'

    """Select Date + Get Data"""
    r = s.post(url, data=data)
    return r
    #soup = BeautifulSoup(r.content, 'lxml')

In [26]:
r = s.get(url)
update_data()

"""Select Price Report"""
r = s.post(url, data=data)
update_data()

"""Select Daily Prices"""
data['__EVENTTARGET'] = 'ctl00$MainContent$Ddl_Rpt_Option0'
data['ctl00$MainContent$Ddl_Rpt_Option0'] = 'Daily Prices'

r = s.post(url, data=data)
update_data()

data['__EVENTTARGET'] = ''
data['ctl00$MainContent$Txt_FrmDate']  = '25/07/2022'
data['ctl00$MainContent$btn_getdata1'] =  'Get Data'

"""Select Date + Get Data"""
r = s.post(url, data=data)
#soup = BeautifulSoup(r.content, 'lxml')

In [31]:
df23=pd.read_html(r.text,header=[1])[0]
df23

,Centre,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
0,CentreRiceWheatAtta (Wheat)Gram DalTur/Arhar D...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Centre,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
2,Average Price,36.83,30.12,33.95,73.59,104.7,104.96,102.09,96.74,41.64,...,157.23,159.52,182.6,137.91,48.82,282.12,19.78,27.27,25.77,35.57
3,Maximum Price,57,45,64,120,135,144,132,130,55,...,227,201,238,187,100,547,29,55,60,80
4,Minimum Price,21,17,22,55,71,73,78,68,35,...,92,124,139,86,33,128,8,14,11,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,Department of Consumer Affairs&nbsp( Price Mon...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,Source:- State civil supplies Dept.Note:- All ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,Source:- State civil supplies Dept.,Note:- All India Average Prices of Oils are on...,NR -&gt Not Reported,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,Developed by:- National Informatics Centre(NIC),Developed by:- National Informatics Centre(NIC),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
r=get_report_html('25/07/2022')
print(r)
#df24=pd.read_html(r.text,header=[1])[0]

TypeError: 'NoneType' object is not subscriptable

In [27]:
#df_list=pd.read_html(r.text,header=[1])
#r=get_report_html('25/07/2022')
df=pd.read_html(r.text,header=[1])[0]
df.drop([0,1],inplace=True)
df1=df[df["Centre"].isin(["EAST ZONE","NORTH ZONE","WEST ZONE","SOUTH ZONE","NORTH EAST ZONE"])]
df.drop(df1.index,inplace=True)
df=df[4:-11]

#replacing NR with np.nan
df=df.replace("NR",np.nan)

In [28]:
#converting the datatype of columns to numeric

#df["Rice"] = pd.to_numeric(df["Rice"])
l=df.columns
for i in range(1,len(l)):
    df[l[i]] = pd.to_numeric(df[l[i]])

In [29]:
df

,Centre,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
7,CHANDIGARH,38.0,24.0,28.0,78.0,113.0,111.0,112.0,90.0,41.0,...,175.0,168.0,170.0,145.0,48.0,229.0,23.0,20.0,30.0,47.0
8,DELHI,32.0,24.0,28.0,70.0,107.0,115.0,100.0,92.0,41.0,...,169.0,153.0,201.0,137.0,57.0,233.0,24.0,28.0,30.0,33.0
9,HISAR,30.0,23.0,26.0,65.0,90.0,90.0,95.0,90.0,39.0,...,140.0,145.0,170.0,130.0,42.0,240.0,23.0,22.0,20.0,28.0
10,KARNAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,PANCHKULA,32.0,26.0,30.0,66.0,96.0,94.0,96.0,87.0,NaN,...,182.0,191.0,209.0,171.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,KARIMNAGAR,35.0,32.0,44.0,70.0,102.0,105.0,104.0,90.0,38.0,...,135.0,NaN,175.0,140.0,48.0,304.0,20.0,25.0,20.0,30.0
191,WARANGAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,ADILABAD,51.0,36.0,41.0,67.0,102.0,98.0,98.0,95.0,37.0,...,165.0,167.0,172.0,133.0,52.0,307.0,21.0,27.0,26.0,30.0
193,SURYAPET,35.0,35.0,28.0,63.0,93.0,95.0,96.0,90.0,47.0,...,163.0,NaN,191.0,150.0,55.0,282.0,20.0,15.0,37.0,62.0


In [133]:
df.describe()

,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,Milk @,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
count,173.000000,156.000000,173.000000,170.000000,170.000000,171.000000,171.000000,173.000000,168.000000,165.000000,...,166.000000,140.000000,154.000000,143.000000,164.000000,162.000000,168.000000,170.000000,170.000000,167.000000
mean,36.832370,30.115385,33.947977,73.594118,104.700000,104.964912,102.093567,96.739884,41.636905,52.333333,...,157.234940,161.500000,184.123377,139.573427,48.823171,282.117284,19.779762,27.270588,25.770588,35.574850
std,8.124088,6.349967,8.379284,9.686504,9.098312,11.618390,10.116528,9.552576,3.124992,8.102494,...,22.709207,15.103718,16.013609,16.499035,8.539289,67.100220,4.468688,7.494990,7.083631,14.106483
min,21.000000,17.000000,22.000000,55.000000,71.000000,73.000000,78.000000,68.000000,35.000000,38.000000,...,92.000000,124.000000,139.000000,86.000000,33.000000,128.000000,8.000000,14.000000,11.000000,10.000000
25%,31.000000,25.000000,28.000000,67.000000,98.000000,97.000000,95.000000,90.000000,40.000000,46.000000,...,139.250000,150.000000,175.000000,129.000000,43.000000,240.000000,16.750000,22.000000,22.000000,26.000000
50%,35.000000,29.000000,31.000000,71.000000,104.000000,103.000000,100.000000,95.000000,41.000000,51.000000,...,159.500000,160.500000,184.000000,138.000000,47.000000,273.000000,21.000000,26.000000,25.000000,35.000000
75%,42.000000,35.000000,36.000000,77.750000,110.000000,115.000000,108.000000,102.000000,43.000000,57.000000,...,172.000000,172.000000,194.000000,149.500000,53.250000,307.000000,23.000000,30.000000,28.000000,43.000000
max,57.000000,45.000000,64.000000,120.000000,135.000000,144.000000,132.000000,130.000000,55.000000,80.000000,...,227.000000,201.000000,238.000000,187.000000,100.000000,547.000000,29.000000,55.000000,60.000000,80.000000


In [135]:
df.to_csv("prices_25_7.csv",index=False)

In [137]:
df1

,Centre,Rice,Wheat,Atta (Wheat),Gram Dal,Tur/Arhar Dal,Urad Dal,Moong Dal,Masoor Dal,Sugar,...,Vanaspati (Packed),Soya Oil (Packed),Sunflower Oil (Packed),Palm Oil (Packed),Gur,Tea Loose,Salt Pack (Iodised),Potato,Onion,Tomato
6,NORTH ZONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,WEST ZONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,EAST ZONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,SOUTH ZONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Timeseries Generation

In [10]:
from datetime import date, timedelta

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [15]:
start_date = date(2013, 1, 1)
end_date = date(2013, 1, 2)
for single_date in daterange(start_date, end_date):
    x=str(single_date.strftime("%d-%m-%Y"))
    print(type(x))
    print(x)

<class 'str'>
01-01-2013
